In [6]:
import os 
import pandas as pd
from tqdm import tqdm

### Help Function

In [ ]:
# For pandas
# Reducing dataframe memory usage :-
def ReduceMemory(df: pd.DataFrame):
    """
    This function reduces the associated dataframe's memory usage.
    It reassigns the data-types of columns according to their min-max values.
    It also displays the dataframe information after memory reduction.
    """;
    
    # Reducing float column memory usage:-
    for col in tqdm(df.iloc[0:2, 1:].select_dtypes('float').columns):
        col_min = np.amin(df[col].dropna());
        col_max = np.amax(df[col].dropna());
        
        if col_min >= np.finfo(np.float16).min and col_max <= np.finfo(np.float16).max: 
            df[col] = df[col].astype(np.float16)
        elif col_min >= np.finfo(np.float32).min and col_max <= np.finfo(np.float32).max : 
            df[col] = df[col].astype(np.float32)
        else: pass;

    # Reducing integer column memory usage:-
    for col in tqdm(df.iloc[0:2, 1:].select_dtypes('int').columns):
        col_min = df[col].min(); 
        col_max = df[col].max();
        
        if col_min >= np.iinfo(np.int8).min and col_max <= np.iinfo(np.int8).max:
            df[col] = df[col].astype(np.int8);
        elif col_min >= np.iinfo(np.int16).min and col_max <= np.iinfo(np.int16).max:
            df[col] = df[col].astype(np.int16);
        elif col_min >= np.iinfo(np.int32).min & col_max <= np.iinfo(np.int32).max:
            df[col] = df[col].astype(np.int32);
        else: pass;
        
    display(df.info()); 
    
    return df;

In [ ]:
input_dir = '/Users/dylan/DylanLi/Code_Repo/CMU18763_Projects1/fifadata'

In [ ]:
csv_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.csv')]

In [ ]:
csv_files

In [ ]:
for file in csv_files: 
    df = pd.read_csv(file)
    df['year'] = int(f'20{file[-6:-4]}')
    df.to_csv(file, index=False)

In [ ]:
df = pd.concat([pd.read_csv(f) for f in csv_files])


In [ ]:
df = ReduceMemory(df)

In [ ]:
# Write the concatenated DataFrame to a new CSV file
output_file = '/Users/dylan/DylanLi/Code_Repo/CMU18763_Projects1/full_data.csv'
df.to_csv(output_file, index=False)

## Using pyspark to read table and write to PostgreSQL

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

appName = "Project1"
master = "local"


sc = SparkSession.builder.appName(appName).getOrCreate()
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession.builder.getOrCreate()

In [ ]:
# read multiple csv files into spark dataframe
df = spark.read.csv(output_file, header=True, inferSchema=True)

In [ ]:
# Insert unique id
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("id", monotonically_increasing_id())

In [ ]:
db_properties={}
db_properties['username']="postgres"
db_properties['password']="010323"
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
db_properties['table']="fifa"
db_properties['driver']="org.postgresql.Driver"

In [ ]:
df.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

In [ ]:
df_read = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()
    
df_read.show()

## Task II 

### Question 1


In [ ]:
df_read.createOrReplaceTempView("df_view")

In [ ]:
sqlWay1 = spark.sql("""
SELECT dv.club_name, COUNT(*) AS player_count
FROM   df_view dv
WHERE  dv.year = 2022 AND dv.club_contract_valid_until = 2023
GROUP BY dv.club_name
ORDER BY player_count DESC
LIMIT 5;

                    """)

In [ ]:
sqlWay1.show()

### Question 2 


In [ ]:
sqlWay2 = spark.sql("""
WITH club_counts AS (
    SELECT dv.club_name, dv.year, COUNT(*) AS player_count
    FROM df_view dv
    WHERE dv.age > 27
    GROUP BY dv.club_name, dv.year
),
club_averages AS (
    SELECT cc.club_name, AVG(cc.player_count) AS average_count
    FROM club_counts cc
    GROUP BY cc.club_name
)
SELECT ca.club_name, ca.average_count
FROM club_averages ca
WHERE (
    SELECT COUNT(*) FROM club_averages ca2 WHERE ca2.average_count > ca.average_count
) < 10
ORDER BY ca.average_count DESC;
""")

In [ ]:
sqlWay2.show()

### Question 3

In [ ]:
sqlWay3 = spark.sql("""
WITH yearly_counts AS (
    SELECT dv.year, dv.nation_position, COUNT(*) AS position_count
    FROM df_view dv
    WHERE dv.nation_position IS NOT NULL
    GROUP BY dv.year, dv.nation_position
),
max_counts AS (
    SELECT yc.year, MAX(yc.position_count) AS max_count
    FROM yearly_counts yc
    GROUP BY yc.year
)
SELECT mc.year, yc.nation_position, mc.max_count
FROM max_counts mc
JOIN yearly_counts yc ON mc.year = yc.year AND mc.max_count = yc.position_count
ORDER BY mc.year;


""")

In [ ]:
sqlWay3.show()

### Task III 


### EDA

#### Data Preprocessing

##### Drop Useless column

In [ ]:
# Drop useless column
useless_columns = ['sofifa_id', 'player_url', 'long_name', 'dob', 'club_loaned_from',
                   'nation_position', 'nation_jersey_number', 'body_type', 'real_face',
                   'player_face_url', 'club_logo_url', 'nation_logo_url', 'nation_flag_url',
                    'goalkeeping_speed', 'player_tags', 'nation_team_id', 'short_name', 'league_name','id', 'club_joined','club_contract_valid_until'] #TODO how to deal with time data

In [ ]:
# if in linux use
# df_read = df

In [ ]:

new_df = df_read.drop(*useless_columns)

In [ ]:
new_df.show(5)

##### Delete columns that include url

In [ ]:
from pyspark.sql.functions import col

# Get a list of columns that include URLs
url_columns = [c for c in new_df.columns if 'url' in c]

# Drop the columns that include URLs
new_df = new_df.drop(*url_columns)

##### Drop Columns that Missing Value are more than 50%

In [ ]:
from pyspark.sql.functions import col

cols_to_drop = []
for i in new_df.columns:
    missing = new_df.filter(col(i).isNull()).count() / new_df.count() * 100
    if missing > 50:
        print('{} - {}%'.format(i, round(missing)))
        cols_to_drop.append(i)

In [ ]:
new_df = new_df.drop(*cols_to_drop)

In [ ]:

from pyspark.sql import DataFrame
from pyspark.sql.functions import col, sum as _sum, when

na_counts = new_df.select([_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in new_df.columns])

In [ ]:
na_counts.show()

##### drop value after +


In [ ]:
from pyspark.sql.functions import split
from pyspark.sql.types import IntegerType

columns1 = ['ls','st','rs','lw','lf','cf','rf','rw','lam','cam','ram',
            'lm','lcm','cm','rcm','rm','lwb','ldm', 'cdm','rdm','rwb',
            'lb','lcb','cb','rcb','rb']

for col in columns1:
    new_df = new_df.withColumn(col, split(new_df[col], '\+').getItem(0).cast(IntegerType()))

In [ ]:
new_df.show(5)

#### Handling Missing Value


In [ ]:
new_df = new_df.fillna(0)

In [ ]:
na_value = "NA"

In [ ]:
string_cols = [c for c, t in new_df.dtypes if t == 'string']

In [ ]:

for col in string_cols:
  new_df = new_df.fillna(na_value, subset=[col])

In [ ]:
new_df.show()

#### Feature Engineering

In [ ]:
from pyspark.sql.functions import split, when, col, array_contains
import itertools

# Split positions into array
split_positions = split(new_df['player_positions'], ', ')  

# Get distinct positions as a list
distinct_positions = list(set(list(itertools.chain(*new_df.select(split_positions.alias('positions')).distinct().rdd.flatMap(lambda x: x).collect()))))

# Create a column for each distinct position
for position in distinct_positions:
  new_df = new_df.withColumn(
    'Position_' + position,
     when(array_contains(split_positions, position), 1).otherwise(0)
  )




In [ ]:
new_df = new_df.drop('player_positions')

In [ ]:
#Check missing values again
na_counts = new_df.select([_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in new_df.columns])
na_counts.show()

In [ ]:
# We are gonna preprocess the preffered_foot using one-hot encoder
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
indexer = StringIndexer(inputCol='preferred_foot', outputCol='indexed_preferred_foot')
encoder = OneHotEncoder(inputCols=['indexed_preferred_foot'], outputCols=['preferred_foot_encoded'])

pipeline = Pipeline(stages=[indexer,encoder])

model = pipeline.fit(new_df)

col_to_drop = ['indexed_preferred_foot','preferred_foot']
data_encoded = model.transform(new_df).drop(*col_to_drop)

In [ ]:
# use label encoder for work_rate and player_positions label_encoder
from pyspark.ml.feature import StringIndexer

# Loop over each string column in the DataFrame
for col_name, data_type in data_encoded.dtypes:
    if data_type == 'string':
        # Create a StringIndexer object and fit it to the column
        indexer = StringIndexer(inputCol=col_name, outputCol=col_name + '_indexed')
        model = indexer.fit(data_encoded)
        
        # Transform the column using the fitted indexer
        data_encoded = model.transform(data_encoded).drop(col_name)

In [ ]:
data_encoded.show(5)

In [ ]:
data_encoded.printSchema()